 #  TargetClass - 25000x -  inference on Validation data - uncertainty assessment
 

## Import libraries

In [ ]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# setup CUDA_VISIBLE DEVICES for titan.sci.utah.edu
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [ ]:
#Import libraries - fastai_v1

from fastai.vision import *
from fastai.metrics import error_rate

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.utils.multiclass import unique_labels

from fastai.basic_train import DatasetType
from fastai.torch_core import to_np
import torch


## I/O and hyper parameters

In [ ]:
# Parameters and hyper-parameters

# CSV file contains test dataset only (synthetic data)
csv_val_FileName = 'Dataset_TargetClass_Overlap-9Blocks_25000xOnly_shuffled_fastai-v1_val.csv'
csv_val = os.path.join('../CSV_InputFiles_TargetClass',csv_val_FileName)

csv_result = os.path.join(os.getcwd(),'Dataset_TargetClass_Overlap-9Blocks_25000xOnly_shuffled_fastai-v1_val-Prediction.csv')
csv_result_Uncertainty = os.path.join(os.getcwd(),'Dataset_TargetClass_Overlap-9Blocks_25000xOnly_shuffled_fastai-v1_val-PredictionWithUncertainty.csv')

csv_result_MajVoting = os.path.join(os.getcwd(),'Dataset_TargetClass_Overlap-9Blocks_25000xOnly_shuffled_fastai-v1_val-PredictionWithUncertainty_MajVoting.csv')
#csv_result_Top2_MajVoting = os.path.join(os.getcwd(),'Dataset_MixedMaterials_ImageClassification_oversample_shuffled_fastai-v1_val-PredictionTop2_MajVoting.csv')

# Network
model_path = os.path.join(os.getcwd(),'models')
model_file = ('TargetClass_fastai-v1_224_all_resnet50.pkl')

# Network architecture
arch = models.resnet50
# Image size
sz = 224
# Batch size
bs = 32
# Default learning rate
lr = 0.01

## Define Validation dataset

In [ ]:
# Read csv file and create dataframe
df_val = pd.read_csv(csv_val, sep=',')
df_val.head()

In [ ]:
df_val.shape

In [ ]:
df_val.groupby(['Label']).size()

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(5,5))
sns.set(style="whitegrid")
sns_plot = sns.countplot(x="Label", data=df_val)
sns_plot.set_xticklabels(sns_plot.get_xticklabels(), rotation=90)
plt.tight_layout()
plt.show()
fig = sns_plot.get_figure()
fig.savefig("BarGraph_Distribution_Label_ValData.png")

## Load CNN Learner

In [ ]:
# Generate validation ImageList
val_ImageList = ImageList.from_csv(os.getcwd(), csv_val_FileName, folder='../Data_TargetClass')

print(val_ImageList)
print(type(val_ImageList))

In [ ]:
# Import CNN model along with wegihts, transforms, and test data)
learn = load_learner(model_path,model_file, test=val_ImageList)

In [ ]:
learn

In [ ]:
learn.data.classes

In [ ]:
# WARNING: incorrect label, as loaded as DatasetType.Test
#learn.data.show_batch(rows=3, ds_type=DatasetType.Test, figsize=(12,9))

## Direct Inference

In [ ]:
# Get direct predictions
y_pred_test, _, losses = learn.get_preds(ds_type=DatasetType.Test,with_loss=True)


In [ ]:
# Generate classes from predictions
y_pred_test_classes = [learn.data.classes[np.argmax(pred)] for pred in y_pred_test]


In [ ]:
learn.data.test_ds.items[0]

In [ ]:
FileNames = ['/'.join(i.split('/', -1)[-4:]) for i in learn.data.test_ds.items]
df_preds_test = pd.DataFrame({'File':FileNames, 'Prediction':y_pred_test_classes})
df_preds_test.head()

In [ ]:
df_preds_test.shape

In [ ]:
# Generate new dataframe result
result = df_val.merge(df_preds_test,on='File',how='left')

In [ ]:
result.shape

In [ ]:
result.head()

In [ ]:
# Save results as CSV file
result.to_csv(csv_result, index=False, na_rep = 'NA')

In [ ]:
# Compute accuracy and confusion matrix


# Generate proper arrays
List_TrueClass_test = result['Label'].tolist()
List_PredClass_test = result['Prediction'].tolist()

# Back to class_nb
List_TrueValue_test = [pd.Index(learn.data.classes).get_loc(x) for x in List_TrueClass_test]
List_PredValue_test = [pd.Index(learn.data.classes).get_loc(x) for x in List_PredClass_test]

# Accuracy score
print("\nAccuracy score:", accuracy_score(List_TrueValue_test, List_PredValue_test))

# Classification report
print("\nClassification report:")
print(classification_report(List_TrueValue_test,List_PredValue_test,target_names=learn.data.classes))


#Confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(List_TrueClass_test,List_PredClass_test, labels=learn.data.classes))



In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    #classes = classes[unique_labels(y_true, y_pred)]
    #classes = [classes[i] for i in unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix')

    #print(cm)

    fig, ax = plt.subplots(1,1,figsize=(8,6))
    plt.grid(False,which='major')
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=90, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    #fig.tight_layout()
    return ax



In [ ]:
ax = plot_confusion_matrix(List_TrueValue_test, List_PredValue_test, learn.data.classes, title='Confusion Matrix - Val data')
plt.tight_layout()
plt.savefig('ConfusionMatrix_ValData.png')

## Additional functions to update model, measure entropy, plot histograms

### Custom dropout

In [ ]:
from typing import Callable

class CustomDropout(nn.Module):
    """Custom Dropout module to be used as a baseline for MC Dropout"""

    def __init__(self, p:float, activate=True):
        super().__init__()
        self.activate = activate
        self.p = p

    def forward(self, x):
        return nn.functional.dropout(x, self.p, training=self.training or self.activate)

    def extra_repr(self):
        return f"p={self.p}, activate={self.activate}"


def switch_custom_dropout(m, activate:bool=True, verbose:bool=False):
    """Turn all Custom Dropouts training mode to true or false according to the variable activate"""
    for c in m.children():
        if isinstance(c, CustomDropout):
            print(f"Current active : {c.activate}")
            print(f"Switching to : {activate}")
            c.activate = activate
        else:
            switch_custom_dropout(c, activate=activate)

def convert_layers(model:nn.Module, original:nn.Module, replacement:nn.Module, get_args:Callable=None,
 additional_args:dict={}):
    """Convert modules of type "original" to "replacement" inside the model
    
    get_args : a function to use on the original module to eventually get its arguements to pass to the new module
    additional_args : a dictionary to add more args to the new module
    """
    for child_name, child in model.named_children():

        if isinstance(child, original):
            # First we grab args from the child
            if get_args:
                original_args = get_args(child)
            else:
                original_args = {}

            # If we want to provide additional args
            if additional_args:
                args = {**original_args, **additional_args}
            else:
                args = original_args

            new_layer = replacement(**args)
            setattr(model, child_name, new_layer)
        else:
            convert_layers(child, original, replacement,
                           get_args, additional_args)

### Metrics

In [ ]:


nb_classes = len(learn.data.classes)

# Note: updated entropy metric with normalization by number of classes
def entropy(probs, softmax=False):
    """Return the prediction of a T*N*C tensor with :
        - T : the number of samples
        - N : the batch size
        - C : the number of classes
    """
    probs = to_np(probs)
    prob = probs.mean(axis=0)

    entrop = - (np.log(prob) * prob).sum(axis=1)
    entrop = entrop / nb_classes
    return entrop


def uncertainty_best_probability(probs):
    """Return standard deviation of the most probable class"""
    idx = probs.mean(axis=0).argmax(axis=1)

    std = probs[:, np.arange(len(idx)), idx].std(axis=0)

    return std


def uncertainty_best_probability2(probs):
    """Return idx, mean and standard deviation of the most probable class"""
    idx = probs.mean(axis=0).argmax(axis=1)
    probs_idx = probs[:, np.arange(len(idx)), idx]
    mean = probs_idx.mean(axis=0)
    std = probs_idx.std(axis=0)
    return idx, mean, std


def BALD(probs):
    """Information Gain, distance between the entropy of averages and average of entropy"""
    entrop1 = entropy(probs)
    probs = to_np(probs)

    entrop2 = - (np.log(probs) * probs).sum(axis=2)
    entrop2 = entrop2.mean(axis=0)
    entrop2 = entrop2 / nb_classes
    
    ig = entrop1 - entrop2
    return ig


def top_k_uncertainty(s, k=5, reverse=True):
    """Return the top k indexes"""
    sorted_s = sorted(list(zip(np.arange(len(s)), s)),
                      key=lambda x: x[1], reverse=reverse)
    output = [sorted_s[i][0] for i in range(k)]
    return output


def get_preds_sample(learn, ds_type=DatasetType.Test, n_sample=10, reduce=None,activ=None,with_loss=False):
    """Get MC Dropout predictions from a learner, and eventually reduce the samples"""
    preds = []
    for i in range(n_sample):
        pred, y = learn.get_preds(ds_type=ds_type, activ=activ)
        pred = pred.view((1,) + pred.shape)
        preds.append(pred)
    preds = torch.cat(preds)
    if reduce == "mean":
        preds = preds.mean(dim=0)
    elif reduce == "std":
        preds = preds.std(dim=0)
    return preds, y


# Warning: only correct for binary classification? need to double-check implementation
def plot_hist_groups(pred,y,metric,bins=None,figsize=(16,16)):
    TP = to_np((pred.mean(dim=0).argmax(dim=1) == y) & (y == 1))
    TN = to_np((pred.mean(dim=0).argmax(dim=1) == y) & (y == 0))
    FP = to_np((pred.mean(dim=0).argmax(dim=1) != y) & (y == 0))
    FN = to_np((pred.mean(dim=0).argmax(dim=1) != y) & (y == 1))
    
    result = metric(pred)
    
    TP_result = result[TP]
    TN_result = result[TN]
    FP_result = result[FP]
    FN_result = result[FN]
    
    fig,ax = plt.subplots(2,2,figsize=figsize)
    
    sns.distplot(TP_result,ax=ax[0,0],bins=bins)
    ax[0,0].set_title(f"True positive")
    
    sns.distplot(TN_result,ax=ax[0,1],bins=bins)
    ax[0,1].set_title(f"True negative")
    
    sns.distplot(FP_result,ax=ax[1,0],bins=bins)
    ax[1,0].set_title(f"False positive")
    
    sns.distplot(FN_result,ax=ax[1,1],bins=bins)
    ax[1,1].set_title(f"False negative")
    
    
def predict_entropy(img,n_times=10):
    pred = learn.predict_with_mc_dropout(img,n_times=n_times)
    probs = [prob[2].view((1,1) + prob[2].shape) for prob in pred]
    probs = torch.cat(probs)
    e = entropy(probs)
    return e

def plot_img_with_entropy(img,n_times=10):
    e = predict_entropy(img,n_times=n_times)
    img = to_np(img.data.permute(1,2,0))
    plt.imshow(img)
    plt.grid(b=None)
    plt.title("Entropy %f:" %(e[0]))
    
def predict_uncertainty(img,n_times=10):
    pred = learn.predict_with_mc_dropout(img,n_times=n_times)
    probs = [prob[2].view((1,1) + prob[2].shape) for prob in pred]
    probs = torch.cat(probs)
    idx, mean, std = uncertainty_best_probability2(probs)
    return idx, mean, std

def plot_img_with_uncertainty(img,n_times=10):
    idx, mean, std = predict_uncertainty(img,n_times=n_times)
    img = to_np(img.data.permute(1,2,0))
    plt.imshow(img)
    plt.grid(b=None)
    plt.title("Inference with uncertainty (label,mean,std): \n %i, %f +- %f " % (idx,mean,std))

### Plot histograms

In [ ]:
# # Generate sample predictions
# preds_sample,y_sample = get_preds_sample(learn)

In [ ]:
# preds_sample.shape

In [ ]:
# # Generate list of ground truth labels from dataframe
# List_StartingMaterials_Val = df_val['StartingMaterial'].tolist()
# y_sample_List = [learn.data.classes.index(label) for label in List_StartingMaterials_Val]
# y_sample = torch.FloatTensor(y_sample_List)

In [ ]:
#plot_hist_groups(preds_sample,y_sample,entropy,bins=20,figsize=(14,14))

### Experiments - uncertainty on single test image

In [ ]:
img = learn.data.test_ds[0][0]

In [ ]:
pred_U = learn.predict_with_mc_dropout(img,n_times=100)
pred_U[:10]


In [ ]:
probs = [prob[2].view((1,1) + prob[2].shape) for prob in pred_U]
probs = torch.cat(probs)
print(probs.shape)
#probs

In [ ]:
# Note: uncertainty enabled by default code
plot_img_with_entropy(img,n_times=100)

In [ ]:
# Note: uncertainty enabled by default code
plot_img_with_uncertainty(img,n_times=100)

In [ ]:
# Note: uncertainty enabled by default code
plot_img_with_uncertainty(img,n_times=200)

### Update model


In [ ]:
# Convert nn.Dropout to CustomDropout module
get_args = lambda dp : {"p" : dp.p}
convert_layers(learn.model,nn.Dropout,CustomDropout,get_args)

# Turn on the stochasticity, I use verbose just to make sure it's working fine
switch_custom_dropout(learn.model,True,verbose=True)

### Inference with uncertainty -  dataset

In [ ]:
# Generate sample predictions
preds_sample_U, _ = get_preds_sample(learn, n_sample=100)

In [ ]:
preds_sample_U.shape

In [ ]:
# Return list of metrics for test dataset: mean, std, entropy
def uncertainty_metrics_sample():
    List_Predidx = []
    List_Predmean = []
    List_Predstd = []
    List_entropy = []
    for i in range(len(learn.data.test_ds)):
            probs = preds_sample_U[:,i,:]
            probs = probs.unsqueeze(1)
            Predidx, Predmean, Predstd = uncertainty_best_probability2(probs)
            e = entropy(probs)
    
            List_Predidx.append(Predidx.numpy()[0])
            List_Predmean.append(Predmean.numpy()[0])
            List_Predstd.append(Predstd.numpy()[0])
            List_entropy.append(e[0])

    return List_Predidx, List_Predmean, List_Predstd, List_entropy


In [ ]:
List_Predidx, List_Predmean, List_Predstd, List_entropy = uncertainty_metrics_sample()

In [ ]:
print(List_Predidx[:10])
print(List_Predmean[:10])
print(List_Predstd[:10])
print(List_entropy[:10])




In [ ]:
# Generate classes from predictions
y_pred_test_classes_U = [learn.data.classes[idx] for idx in List_Predidx]
print(y_pred_test_classes_U[:10])

In [ ]:
# Generate dataframe and save spreadsheet from all samples

FileNames = ['/'.join(i.split('/', -1)[-4:]) for i in learn.data.test_ds.items]

df_preds_test_U = pd.DataFrame({'File':FileNames, 'Prediction':y_pred_test_classes_U,'Pred_Mean':List_Predmean, 'Pred_Std':List_Predstd, 'Entropy':List_entropy})
df_preds_test_U.head()

In [ ]:
# Combine data frames (to double check proper order)
result_U = df_val.merge(df_preds_test_U,on='File',how='left')
df_preds_test_U.shape, result_U.shape

In [ ]:
result_U.head()

In [ ]:
# Save spreadsheet from uncertainty
result_U.to_csv(csv_result_Uncertainty, index=False, na_rep = 'NA',float_format='%.6f')

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    #classes = classes[unique_labels(y_true, y_pred)]
    #classes = [classes[i] for i in unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix')

    #print(cm)

    fig, ax = plt.subplots(1,1,figsize=(8,6))
    plt.grid(False,which='major')
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=90, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    #fig.tight_layout()
    return ax



In [ ]:
# Compute accuracy and confusion matrix


# Generate proper arrays
List_TrueClass_test_U = result_U['Label'].tolist()
List_PredClass_test_U = result_U['Prediction'].tolist()

# Back to class_nb
List_TrueValue_test_U = [pd.Index(learn.data.classes).get_loc(x) for x in List_TrueClass_test_U]
List_PredValue_test_U = [pd.Index(learn.data.classes).get_loc(x) for x in List_PredClass_test_U]

# Accuracy score
print("\nAccuracy score:", accuracy_score(List_PredValue_test_U,List_TrueValue_test_U))

#Confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(List_TrueClass_test_U,List_PredClass_test_U, labels=learn.data.classes))

In [ ]:
ax = plot_confusion_matrix(List_TrueValue_test_U, List_PredValue_test_U, learn.data.classes, title='Confusion Matrix - Val data - Uncertainty')
plt.tight_layout()
plt.savefig('ConfusionMatrix_ValData-Uncertainty.png')

In [ ]:
learn.data.classes

In [ ]:
result_U_StartingMaterial_mean = result_U.groupby('Label')['Pred_Mean','Pred_Std','Entropy'].mean()
#result_U_StartingMaterial_mean = result_U_StartingMaterial_mean.reindex(classes_Labels_ordered)
result_U_StartingMaterial_mean

In [ ]:
result_U_StartingMaterial_std = result_U.groupby('Label')['Pred_Mean','Pred_Std','Entropy'].std()
#result_U_StartingMaterial_std = result_U_StartingMaterial_std.reindex(classes_Labels_ordered)
result_U_StartingMaterial_std

In [ ]:
result_U.groupby('Label').describe(percentiles=[])

In [ ]:
# # Plot histograms of entropy for each starting material

# def plot_hist_metric(df,metric,bins=10,figsize=(16,16)):
    
#     List_25ADU75UO4 = df[df['StartingMaterial']=='25ADU-75UO4'][metric].tolist()
#     List_50ADU50UO4 = df[df['StartingMaterial']=='50ADU-50UO4'][metric].tolist()
#     List_75ADU25UO4 = df[df['StartingMaterial']=='75ADU-25UO4'][metric].tolist()
#     List_ADU = df[df['StartingMaterial']=='ADU'][metric].tolist()
#     List_UO4 = df[df['StartingMaterial']=='UO4-2H2O'][metric].tolist()
    
    
#     fig,ax = plt.subplots(3,2,figsize=figsize)
#     #plt.xlim(0, 0.25)
    
#     #sns.distplot(List_25ADU75UO4,ax=ax[0,0],bins=bins,hist_kws={'range': (0.0, 0.25)},kde_kws={'clip': (0.0, 0.25)})
#     sns.distplot(List_25ADU75UO4,ax=ax[0,0],bins=bins)
#     ax[0,0].set_title(f"List_25ADU75UO4")
#     ax[0,0].set_xlim([-0.05, 0.25])
    
#     sns.distplot(List_50ADU50UO4,ax=ax[0,1],bins=bins)
#     ax[0,1].set_title(f"List_50ADU50UO4")
#     ax[0,1].set_xlim([-0.05, 0.25])
    
#     sns.distplot(List_75ADU25UO4,ax=ax[1,0],bins=bins)
#     ax[1,0].set_title(f"List_75ADU25UO4")
#     ax[1,0].set_xlim([-0.05, 0.25])
    
#     sns.distplot(List_ADU,ax=ax[1,1],bins=bins)
#     ax[1,1].set_title(f"List_ADU")
#     ax[1,1].set_xlim([-0.05, 0.25])
    
#     sns.distplot(List_UO4,ax=ax[2,0],bins=bins,hist_kws={'range': (0.0, 0.25)},kde_kws={'clip': (0.0, 0.25)})
#     ax[2,0].set_title(f"List_UO4")
#     ax[2,0].set_xlim([-0.05, 0.25])
    
    
#     # Save the full figure...
#     #fig.savefig('Histogram_Entropy.png')

In [ ]:
# fig = plot_hist_metric(result_U,"Entropy")
# plt.savefig("Screenshot_Histograms_Label-ValData_Entropy.png")

In [ ]:
# Use random image to assess larger uncertainty...

## Majority Voting


In [ ]:
result_TrueLabel_MajVoting = result.groupby(['Acquisition'])['StartingMaterial'].apply(lambda x: x.mode()[0]).reset_index(name='TrueLabel_MajVoting')
result_TrueLabel_MajVoting.head(15)

In [ ]:
#result.groupby(['Acquisition'])['Prediction'].apply(lambda x: x.mode())

In [ ]:
result_PredLabel_MajVoting = result.groupby(['Acquisition'])['Prediction'].apply(lambda x: x.mode()[0]).reset_index(name='PredLabel_MajVoting')
result_PredLabel_MajVoting.head(15)

In [ ]:
# Combine data frames (to double check proper order)
result_MajVoting = pd.merge(result_TrueLabel_MajVoting, result_PredLabel_MajVoting, how='left', on='Acquisition')
result_MajVoting.head()


In [ ]:
result_MajVoting.to_csv(csv_result_MajVoting, index=False, na_rep = 'NA')

In [ ]:
result_MajVoting.shape

In [ ]:
# Generete proper arrays
List_TrueClass_MajVoting_test = result_MajVoting['TrueLabel_MajVoting'].tolist()
List_PredClass_MajVoting_test = result_MajVoting['PredLabel_MajVoting'].tolist()

# Back to class_nb
List_TrueValue_MajVoting_test = [pd.Index(learn.data.classes).get_loc(x) for x in List_TrueClass_MajVoting_test]
List_PredValue_MajVoting_test = [pd.Index(learn.data.classes).get_loc(x) for x in List_PredClass_MajVoting_test]

In [ ]:
accuracy_score(List_PredValue_MajVoting_test,List_TrueValue_MajVoting_test)

In [ ]:
confusion_matrix(List_TrueClass_MajVoting_test,List_PredClass_MajVoting_test, labels=learn.data.classes)

In [ ]:
print(classification_report(List_TrueValue_MajVoting_test,List_PredValue_MajVoting_test,target_names=learn.data.classes))
